In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout,LeakyReLU,GRU,LSTM,Concatenate,BatchNormalization,Bidirectional,Input
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn import metrics
import itertools
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

## Data Loading

In [ ]:
dataPath = 'data/'

In [ ]:
##Original Dataset is available as a CSV. This is a slightly processed version to improve access time and reduce training latency

df = pd.read_parquet('data/arp_mitm_dataset.parquet')


In [ ]:
df_labels = pd.read_csv("data/ARP_MitM_labels.csv",dtype={"": int, "x": 'float64'})
df_labels = df_labels["x"]

In [ ]:
df.head()

In [ ]:
best_features=df.values
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
df_pre=sc_x.fit_transform(best_features)

print('=====Data Preprocessing=========')
print(df_pre)

In [ ]:
from sklearn.model_selection import train_test_split
y=df_labels
y=np.ravel(y)
print(df_pre.shape)
print(y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(df_pre, y, test_size =0.2)

In [ ]:
##LSTM Based Model

In [ ]:
AX_train=X_train
AX_test=X_test
AY_train=Y_train
AY_test=Y_test
X_train1 = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test1 = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1],1), return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(units = 2, activation='softmax'))
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model_history = model.fit(X_train1, Y_train, epochs=2, batch_size = 256, validation_data = (X_test1, Y_test))
model.save('models/LSTM.h5')

In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
##Simple GRU Model - Same hidden layers as the LSTM

In [ ]:
AX_train=X_train
AX_test=X_test
AY_train=Y_train
AY_test=Y_test
X_train1 = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test1 = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
model = Sequential()
model.add(GRU(50, input_shape=(X_train.shape[1],1), return_sequences=True))
model.add(GRU(50, return_sequences=True))
model.add(GRU(50, return_sequences=True))
model.add(GRU(50))
model.add(Dense(units = 2, activation='softmax'))
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model_history = model.fit(X_train1, Y_train, epochs=2, batch_size = 256, validation_data = (X_test1, Y_test))
model.save('models/GRU_base.h5')


In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
##GRU Model Testing - Trying various combinations of Hidden Layers and testing tuning methods like Dropout

In [ ]:
AX_train=X_train
AX_test=X_test
AY_train=Y_train
AY_test=Y_test
X_train1 = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test1 = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
model = Sequential()
model.add(GRU(50, input_shape=(X_train.shape[1],1), return_sequences=True))
model.add(GRU(25, return_sequences=True))
model.add(GRU(25, return_sequences=True))
model.add(GRU(5))
model.add(Dense(units = 2, activation='softmax'))
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model_history = model.fit(X_train1, Y_train, epochs=5, batch_size = 256, validation_data = (X_test1, Y_test))
model.save('models/GRU_2.h5')


In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
##Base GRU with Dropout

In [ ]:
AX_train=X_train
AX_test=X_test
AY_train=Y_train
AY_test=Y_test
X_train1 = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test1 = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
model = Sequential()
model.add(GRU(50, input_shape=(X_train.shape[1],1), return_sequences=True))
model.add(GRU(50, return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(50, return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(50))
model.add(Dense(units = 2, activation='softmax'))
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model_history = model.fit(X_train1, Y_train, epochs=5, batch_size = 256, validation_data = (X_test1, Y_test))
model.save('models/GRU_3.h5')


In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
##Reduced GRU with Dropout

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

AX_train=X_train
AX_test=X_test
AY_train=Y_train
AY_test=Y_test
X_train1 = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test1 = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
model = Sequential()
model.add(GRU(50, input_shape=(X_train.shape[1],1), return_sequences=True))
model.add(GRU(25, return_sequences=True))
model.ad
model.add(GRU(25, return_sequences=True))
model.add(GRU(5))
model.add(Dense(units = 2, activation='softmax'))
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model_history = model.fit(X_train1, Y_train, epochs=5, batch_size = 256, validation_data = (X_test1, Y_test))
model.save('models/GRU_4.h5')


In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.show()